<a href="https://colab.research.google.com/github/PrasoonPratham/tensorflow-etl-benchmark/blob/main/ETL_Benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
import tensorflow_datasets as tdfs
import tensorflow_addons as tfa

In [2]:
data = tdfs.load("malaria", split='train', as_supervised=True)

2022-06-22 00:55:29.738636: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 1/1 [00:34<00:00, 34.52s/ url]

Dl Completed...: 100%|██████████| 1/1 [00:39<00:00, 39.39s/ url]
                                                                        

Dataset malaria downloaded and prepared to ~/tensorflow_datasets/malaria/1.0.0. Subsequent calls will reuse this data.
Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-06-22 00:56:17.265344: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-22 00:56:17.265505: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
# Importing the inception V3 model
from tensorflow.keras.applications.inception_v3 import InceptionV3
# Downloading the weights
import urllib

weights = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

weights_file = 'inception_v3.h5'
urllib.request.urlretrieve(weights, weights_file)
# Loading the weights
pretrained_model = InceptionV3(
    input_shape=(100, 100, 3),
    include_top=False,
    weights=None
)

pretrained_model.load_weights(weights_file)
pretrained_model.summary()
for layer in pretrained_model.layers:
    layer.trainable = False

last_layer = pretrained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output
from tensorflow.keras.layers import Flatten, Dense

x = Flatten()(last_output)
x = Dense(1024, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)
from tensorflow.keras import Model

model = Model(pretrained_model.input, x)
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [39]:
from tensorflow.keras import layers, Model, Input

input_layer = Input(shape=(100, 100, 3), name="img")
x = layers.Conv2D(16, (3, 3), activation='relu')(input_layer)
x = layers.MaxPooling2D(2, 2)(x)
x = layers.Conv2D(32,(3,3), activation="relu")(x)
x = layers.MaxPooling2D(2, 2)(x)
x = layers.Conv2D(64,(3,3), activation="relu")(x)
x = layers.MaxPooling2D(2, 2)(x)
x = layers.Conv2D(64,(3,3), activation="relu")(x)
x = layers.MaxPooling2D(2, 2)(x)
x = layers.Conv2D(64,(3,3), activation="relu")(x)
x = layers.MaxPooling2D(2, 2)(x)
x = layers.Flatten()(x)
x = layers.Dense(512, activation="relu")(x)
output_layer = layers.Dense(1, activation="sigmoid")(x)

model = Model(input_layer, output_layer, name="CNN")
model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 img (InputLayer)            [(None, 100, 100, 3)]     0         
                                                                 
 conv2d_138 (Conv2D)         (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d_47 (MaxPoolin  (None, 49, 49, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_139 (Conv2D)         (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_48 (MaxPoolin  (None, 23, 23, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_140 (Conv2D)         (None, 21, 21, 64)        18496   

In [20]:
def augmentation(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (100, 100))
    image = image / 255
    image = tf.image.random_flip_left_right(image)
    image = tfa.image.rotate(image, 40, interpolation='NEAREST')
    image = tf.image.random_brightness(image, 0.1)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    return image, label

In [21]:
train = data.map(augmentation)
train_batches = train.shuffle(100).batch(32)

In [22]:
# Training the model
history = model.fit(
    train_batches,
    epochs=15
)

Epoch 1/15


2022-06-22 01:54:50.889020: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


862/862 [==============================] - 22s 24ms/step - loss: 0.5902 - accuracy: 0.6821
Epoch 2/15
862/862 [==============================] - 20s 23ms/step - loss: 0.3239 - accuracy: 0.8657
Epoch 3/15
862/862 [==============================] - 20s 23ms/step - loss: 0.2117 - accuracy: 0.9235
Epoch 4/15
862/862 [==============================] - 20s 23ms/step - loss: 0.1833 - accuracy: 0.9367
Epoch 5/15
862/862 [==============================] - 20s 23ms/step - loss: 0.1712 - accuracy: 0.9408
Epoch 6/15
862/862 [==============================] - 20s 23ms/step - loss: 0.1619 - accuracy: 0.9442
Epoch 7/15
862/862 [==============================] - 20s 23ms/step - loss: 0.1577 - accuracy: 0.9458
Epoch 8/15
862/862 [==============================] - 20s 23ms/step - loss: 0.1514 - accuracy: 0.9472
Epoch 9/15
862/862 [==============================] - 20s 23ms/step - loss: 0.1493 - accuracy: 0.9491
Epoch 10/15
862/862 [==============================] - 20s 23ms/step - loss: 0.1448 - accurac

In [23]:
file_pattern = f'~/tensorflow_datasets/malaria/1.0.0/malaria-train.tfrecord*'
file = tf.data.Dataset.list_files(file_pattern)

In [24]:
train_dataset = file.interleave(
    tf.data.TFRecordDataset,
    cycle_length=4,
    num_parallel_calls=tf.data.experimental.AUTOTUNE
    )

In [25]:
def augmentationV2(serialized_data):
    description ={
        'image': tf.io.FixedLenFeature((), tf.string),
        'label': tf.io.FixedLenFeature((), tf.int64, -1)
    }
    example = tf.io.parse_single_example(serialized_data, description)
    image = tf.image.decode_jpeg(example['image'], channels=3)
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (100, 100))
    image = image / 255
    image = tf.image.random_flip_left_right(image)
    image = tfa.image.rotate(image, 40, interpolation='NEAREST')
    image = tf.image.random_brightness(image, 0.1)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    return image, example['label']

In [26]:
import multiprocessing
cores = multiprocessing.cpu_count()

print(cores)

8


In [ ]:
train_dataset = train_dataset.map(augmentationV2, num_parallel_calls=cores)
train_dataset = train_dataset.cache()

In [28]:
train_dataset = train_dataset.shuffle(100).batch(32)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [30]:
# Training the model
history = model.fit(
    train_dataset,
    epochs=15
)

Epoch 1/15
862/862 [==============================] - 16s 18ms/step - loss: 0.1325 - accuracy: 0.9548
Epoch 2/15
862/862 [==============================] - 14s 16ms/step - loss: 0.1289 - accuracy: 0.9551
Epoch 3/15
862/862 [==============================] - 13s 15ms/step - loss: 0.1251 - accuracy: 0.9570
Epoch 4/15
862/862 [==============================] - 13s 16ms/step - loss: 0.1218 - accuracy: 0.9590
Epoch 5/15
862/862 [==============================] - 13s 16ms/step - loss: 0.1182 - accuracy: 0.9600
Epoch 6/15
862/862 [==============================] - 13s 15ms/step - loss: 0.1157 - accuracy: 0.9602
Epoch 7/15
862/862 [==============================] - 14s 16ms/step - loss: 0.1130 - accuracy: 0.9611
Epoch 8/15
862/862 [==============================] - 13s 16ms/step - loss: 0.1095 - accuracy: 0.9632
Epoch 9/15
862/862 [==============================] - 14s 16ms/step - loss: 0.1070 - accuracy: 0.9635
Epoch 10/15
862/862 [==============================] - 13s 16ms/step - loss: 0.104